### Imports

In [1]:
import pandas as pd 
import os
%matplotlib inline
   
from os import listdir
from utils import *
from tensorflow.keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential

2022-04-28 17:32:56.686181: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pedrod33/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-28 17:32:56.686222: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Loading Images

In [2]:
image_dir="/home/pedrod33/Desktop/MRSI/ano1s2/AA/Brain_Tumor_Detection/archive/brain_tumor_dataset/"

### Making directory for augmented images

**A directory is formed using os.makedirs() function for augmented images(yes/ no). Note- custom directory is obtained in outputs folder.**

In [3]:

#os.makedirs('../output/kaggle/working/augmented-images')
#os.makedirs('../output/kaggle/working/augmented-images/yes')
#os.makedirs('../output/kaggle/working/augmented-images/no')

## Augmentation of images 
**About the data:
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous andno contains 98 Brain MRI Images that are non-tumorous.**

In [4]:
IMG_SIZE = 164
X, y = read_images()
X = crop_images(X)
X = resize_and_rescale(X, 164)
X_train, y_train, X_test, y_test = split_data(X, y)
X_train, y_train = augment_data(X_train,y_train, 164)

X_train = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_train])
X_test = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_test])

X_train, y_train, X_val, y_val = split_data(X_train, y_train,0.25)

2022-04-28 17:32:58.777303: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-28 17:32:58.777335: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pedrod33): /proc/driver/nvidia/version does not exist
2022-04-28 17:32:58.777600: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Visualization of data

In [5]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of validation examples = " + str(X_val.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))

number of training examples = 454
number of validation examples = 152
number of test examples = 51


In [11]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(input_shape)) 
    model.add(ZeroPadding2D((2, 2))) 
    #Conv2D(32, (7, 7), strides = (1, 1))(X_input)
    model.add(Conv2D(filters=32, kernel_size=4, input_shape=input_shape, strides = (1, 1)))
    model.add(BatchNormalization(axis = 3, name = 'bn0'))
    model.add(Activation('leaky_relu')) 
    
    
    model.add(MaxPooling2D((4, 4)))
    model.add(MaxPooling2D((4, 4))) 
    model.add(Dropout(.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid')) 
    
    return model

In [12]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE,3)
model=build_model(IMG_SHAPE)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_1 (ZeroPaddi  (None, 168, 168, 3)      0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 165, 165, 32)      1568      
                                                                 
 bn0 (BatchNormalization)    (None, 165, 165, 32)      128       
                                                                 
 activation_1 (Activation)   (None, 165, 165, 32)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 41, 41, 32)       0         
 2D)                                                             
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 10, 10, 32)      

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x=X_train, y=y_train, batch_size=32, epochs=22, validation_data=(X_val, y_val))

Epoch 1/22
15/15 [==============================] - 9s 565ms/step - loss: 1.0921 - accuracy: 0.5551 - val_loss: 0.6975 - val_accuracy: 0.5658
Epoch 2/22
15/15 [==============================] - 8s 553ms/step - loss: 0.7843 - accuracy: 0.6256 - val_loss: 0.6803 - val_accuracy: 0.5658
Epoch 3/22
15/15 [==============================] - 9s 597ms/step - loss: 0.7208 - accuracy: 0.6784 - val_loss: 0.6665 - val_accuracy: 0.5658
Epoch 4/22
15/15 [==============================] - 9s 586ms/step - loss: 0.5348 - accuracy: 0.7599 - val_loss: 0.6541 - val_accuracy: 0.5724
Epoch 5/22
15/15 [==============================] - 9s 588ms/step - loss: 0.5603 - accuracy: 0.7093 - val_loss: 0.6431 - val_accuracy: 0.5921
Epoch 6/22
15/15 [==============================] - 9s 599ms/step - loss: 0.4964 - accuracy: 0.7687 - val_loss: 0.6367 - val_accuracy: 0.6184
Epoch 7/22
15/15 [==============================] - 9s 598ms/step - loss: 0.4830 - accuracy: 0.7665 - val_loss: 0.6240 - val_accuracy: 0.6579
Epoch 

### Plotting of accuracy

In [10]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 116ms/step - loss: 0.5135 - accuracy: 0.7843


[0.5134623646736145, 0.7843137383460999]